In [ ]:
import pandas as pd
import os 

In [ ]:
downloads_23andME = '/Users/jerenolsen/Desktop/genome_downloads/23andMe_final_dir'
#702 files originally

In [ ]:
def get_invalid_files(folder):

    headers = []
    failed_files = []
    filepaths = get_file_paths(folder)
    
    #Try reading 
    for path in filepaths:
        header = read_header(path)
        if header:
            headers.append(header)
        else:
            failed_files.append(path.split('/')[-1])
            
    for header in headers:
        #Check for 23andMe header
        if not check_23andMe(header):
            failed_files.append(header[0])
            
        #Check for build
        if not check_build(header):
            failed_files.append(header[0])
            
        #Check Columns
        if not check_columns(header):
            failed_files.append(header[0])
        
    failed_files = list(set(failed_files))
    passed_headers = [header for header in headers if header[0] not in failed_files]
            
    return passed_headers, failed_files
    

In [ ]:
def read_header(path_23andMe):
    header = []
    with open(path_23andMe, 'r') as f:
        header.append(path_23andMe.split('/')[-1])
        try:
            for line in f:
                if line[0] == '#':
                    header.append(line)
                else:
                    break
        except Exception as e:
            return False
                
    return header

def get_file_paths(folder):
    paths = [path for path in os.listdir(folder)]
    paths = [os.path.join(folder, path) for path in paths]
    return paths

def check_23andMe(header):
    try:
        if len(header) == 1 or 'generated by 23andMe' not in header[1]:
            return False
    except Exception as e:
        return False
    
    return True

def check_build(header):
    build_lines = [line for line in header if 'build' in line]
    build_lines = [line for line in build_lines if 'we are using' in line.lower()]
    if build_lines == []:
        return False
    return True

def check_columns(header):
    cols = header[-1]
    if 'rsid' not in cols:
        return False
    return True


In [ ]:
passed_headers, invalid_files = get_invalid_files(downloads_23andME)

In [ ]:
def remove_invalid_files(file_dir, invalid_files):
    for file in invalid_files:
        path = os.path.join(file_dir,file)
        os.remove(path)
        
    return True
        

In [ ]:
remove_invalid_files(downloads_23andME, invalid_files)

In [ ]:
### FIND ALL BUILD 36 23andME FILES ###

In [ ]:
import os

In [ ]:
dir_path = '/Users/jerenolsen/Desktop/genome_downloads/23andMe_final_dir'

In [ ]:
b36_files = []
for file in os.listdir(dir_path):
    try:
        f_path = os.path.join(dir_path, file)
        build = None
        header_lines = []
        with open(f_path, 'r') as f:
            for line in f:
                if line[0] == '#':
                    header_lines.append(line)
                else:
                    break

        for line in header_lines:
            if 'build' in line and '36' in line:
                print(f"file: {file} is build 36")
                b36_files.append(file)
                continue
    except Exception as e:
        print(f"file {file} failed to read")
            

In [ ]:
len(set(b36_files))

In [ ]:
profiles = [f.split('_')[0] for f in b36_files]
profiles

In [ ]:
GCP_files_issue = ['23andMe_'+profile+'_final.vcf' for profile in profiles]

In [ ]:
GCP_files_issue

In [ ]:
GCP_final_output = []
with open('/Users/jerenolsen/desktop/gcp_final_output.txt', 'r') as f:
    for line in f:
        GCP_final_output.append(line.strip('\n'))
        
ok_to_keep = ['input_genomes/huFE1569_23andMe_genome.txt', 'input_genomes/huFF6AB4_23andMe_genome.txt', 'input_genomes/huFFAD87_23andMe_genome.txt']
for item in ok_to_keep:
    item = item.split("/")[-1].split("_")[0]
    GCP_final_output.append('23andMe_'+item+'_final.vcf')

In [ ]:
GCP_final_output = [item.split('/')[-1] for item in GCP_final_output]

In [ ]:
print(len(GCP_final_output))

In [ ]:
GCP_final_output

In [ ]:
remove_files = [file for file in GCP_files_issue if file in GCP_final_output]

In [ ]:
len(remove_files)


In [ ]:
remove_files = set(remove_files)
remove_files = list(remove_files)

In [ ]:
len(remove_files)

In [ ]:
remove_files

In [ ]:
for file in remove_files:
    print(f"gsutil rm gs://hvd-pgp-genomics-files/final_output/{file}")

In [ ]:
ok_to_keep = ['input_genomes/huFE1569_23andMe_genome.txt', 'input_genomes/huFF6AB4_23andMe_genome.txt', 'input_genomes/huFFAD87_23andMe_genome.txt']

In [ ]:
len(set(profiles))

In [ ]:
b36_files = list(set(b36_files))

In [ ]:
ok_to_keep = [f.split("/")[-1] for f in ok_to_keep]

In [ ]:
ok_to_keep

In [ ]:
print([item for item in ok_to_keep if item in b36_files])